In [13]:
import numpy as np
import pandas as pd
from tabulate import tabulate
# Loan with varying interest rates and 5% decreasing payments
# Timeline: t = 0,...,27


In [14]:
# Question 2

In [2]:
# Interest rates by year:
# - 2.5% for the first 5 years
# - 3.5% for the next 5 years
# - 4% for the last 17 years
i = np.array([0.025]*5 + [0.035]*5 + [0.04]*17)  # length = 27

# Discount factors v_t
# v_0 = 1
# v_t = product over j=1..t of 1/(1+i_j)
discount_factor = np.concatenate(([1.0], np.cumprod(1/(1+i))))


In [3]:
# Cash inflows:
# - 30,000 EUR at t=0
# - 10,000 EUR at t=1
# - 15,000 EUR at t=2
PV = 30000 * discount_factor[0] + 10000 * discount_factor[1] + 15000 * discount_factor[2]


In [4]:
# 25 yearly payments from t=3 to t=27
# Each year payment decreases by 5%

pay_ratio = (0.95) ** np.arange(25)   # A, A*0.95, A*0.95^2, ...

# S = weighted sum of discount factors
S = np.sum(pay_ratio * discount_factor[3:28])

# Solve for first payment A
A = PV / S


In [5]:
years = np.arange(3, 28)        # payment years
K = -A * pay_ratio              # negative = outflow for Cynthia

payments_df = pd.DataFrame({
    "Year": years,
    "Payment": K,
    "DiscountFactor": discount_factor[3:28],
    "PV Contribution": (-K) * discount_factor[3:28]
})


In [6]:
pv_payments = np.sum((-K) * discount_factor[3:28])

check = pd.Series({
    "PV_disbursements": PV,
    "PV_payments": pv_payments,
    "Difference": pv_payments - PV
})


In [12]:
print(f"First payment A at t=3: {A:,.2f} EUR")
print(f"Total PV of disbursements: {PV:,.2f} EUR")
print(f"Total PV of payments:      {pv_payments:,.2f} EUR")
print(f"Difference:                {pv_payments - PV:.6f} EUR")

print("\nPayment Schedule:")
print(tabulate(payments_df_rounded, headers="keys", tablefmt="compact", showindex=False))



First payment A at t=3: 5,402.26 EUR
Total PV of disbursements: 54,033.31 EUR
Total PV of payments:      54,033.31 EUR
Difference:                -0.000000 EUR

Payment Schedule:
  Year    Payment    DiscountFactor    PV Contribution
------  ---------  ----------------  -----------------
     3   -5402.26              0.93            5016.54
     4   -5132.15              0.91            4649.47
     5   -4875.54              0.88            4309.27
     6   -4631.76              0.85            3955.37
     7   -4400.18              0.83            3630.53
     8   -4180.17              0.8             3332.37
     9   -3971.16              0.77            3058.7
    10   -3772.6               0.74            2807.5
    11   -3583.97              0.72            2564.54
    12   -3404.77              0.69            2342.61
    13   -3234.53              0.66            2139.89
    14   -3072.81              0.64            1954.7
    15   -2919.17              0.61            1785.55

In [15]:
# Question 3

In [21]:
# CNN 
# Your estimated monthly payments are $1,225.63 (including taxes, insurance, and PMI if applicable), and you will pay $70,613 in interest over the life of the loan. Loan Information
# Loan amount	$150,000
# Annual interest rate	5.500%
# Number of months	180
# Monthly principal and interest payment	$1,225.63
# Monthly property taxes	$.00
# Monthly hazard insurance	$.00
# Monthly PMI (if applicable)	$.00
# Total monthly payment (including taxes, insurance, and PMI if applicable)	$1,225.63

In [22]:
def declining_annuity_schedule(disbursements, rates_by_year, n_payments, start_t, annual_decay):
    i = np.array(rates_by_year, dtype=float)
    discount_factor = np.concatenate(([1.0], np.cumprod(1/(1+i))))
    PV = sum(amount * discount_factor[t] for t, amount in disbursements)
    pay_ratio = (1 - annual_decay) ** np.arange(n_payments)
    S = np.sum(pay_ratio * discount_factor[start_t:start_t + n_payments])
    A = PV / S
    years = np.arange(start_t, start_t + n_payments)
    K = -A * pay_ratio
    payments_df = pd.DataFrame({
        "Year": years,
        "Payment": K,
        "DiscountFactor": discount_factor[start_t:start_t + n_payments],
        "PV Contribution": (-K) * discount_factor[start_t:start_t + n_payments]
    })
    pv_payments = payments_df["PV Contribution"].sum()
    summary = {
        f"First payment A at t={start_t}": A,
        "Total PV of disbursements": PV,
        "Total PV of payments": pv_payments,
        "Difference": pv_payments - PV
    }
    payments_df_rounded = payments_df.copy()
    payments_df_rounded["Payment"] = payments_df_rounded["Payment"].round(2)
    payments_df_rounded["DiscountFactor"] = payments_df_rounded["DiscountFactor"].round(6)
    payments_df_rounded["PV Contribution"] = payments_df_rounded["PV Contribution"].round(2)
    print(f"First payment A at t={start_t}: {A:,.2f} EUR")
    print(f"Total PV of disbursements: {PV:,.2f} EUR")
    print(f"Total PV of payments:      {pv_payments:,.2f} EUR")
    print(f"Difference:                {pv_payments - PV:.6f} EUR")
    print("\nPayment Schedule:")
    print(tabulate(payments_df_rounded, headers="keys", tablefmt="compact", showindex=False))
    return A, payments_df, summary

# ---- Scenario: Girona castle purchase ----
price = 10_750_000.0
down_payment_pct = 0.30
loan_term_years = 25
fixed_annual_rate = 0.035

loan_amount = price * (1 - down_payment_pct)
down_payment = price - loan_amount

print(f"Property price: {price:,.2f} EUR")
print(f"Down payment ({down_payment_pct:.0%}): {down_payment:,.2f} EUR")
print(f"Loan amount: {loan_amount:,.2f} EUR\n")

A_year, df_year, summary = declining_annuity_schedule(
    disbursements=[(0, loan_amount)],
    rates_by_year=[fixed_annual_rate]*loan_term_years,
    n_payments=loan_term_years,
    start_t=1,
    annual_decay=0.0
)

r_m = fixed_annual_rate/12
n_m = loan_term_years*12
pmt_month = loan_amount * r_m / (1 - (1 + r_m)**(-n_m))
print(f"\nEquivalent constant monthly payment: {pmt_month:,.2f} EUR")


Property price: 10,750,000.00 EUR
Down payment (30%): 3,225,000.00 EUR
Loan amount: 7,525,000.00 EUR

First payment A at t=1: 456,572.12 EUR
Total PV of disbursements: 7,525,000.00 EUR
Total PV of payments:      7,525,000.00 EUR
Difference:                0.000000 EUR

Payment Schedule:
  Year    Payment    DiscountFactor    PV Contribution
------  ---------  ----------------  -----------------
     1    -456572          0.966184             441132
     2    -456572          0.933511             426215
     3    -456572          0.901943             411802
     4    -456572          0.871442             397876
     5    -456572          0.841973             384421
     6    -456572          0.813501             371422
     7    -456572          0.785991             358862
     8    -456572          0.759412             346726
     9    -456572          0.733731             335001
    10    -456572          0.708919             323673
    11    -456572          0.684946             3127